In [1]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup
from IPython.display import clear_output
from time import sleep
from urllib.request import urlopen, Request

### Web Scraping

In [10]:
def decode_html(raw_html):
    html = raw_html.decode('utf-8')
    return ' '.join(html.split()).replace('> <', '><')


# Make HTML request and decode it
def prepare_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " \
                      "AppleWebKit/537.36 (KHTML, like Gecko) " \
                      "Chrome/76.0.3809.100 Safari/537.36"
    }

    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()
    decoded_html = decode_html(html)

    return BeautifulSoup(decoded_html, 'html.parser')

In [5]:
# Get all important variables for a process
def find_values(url):
    soup = prepare_soup(url)
    content = {}

    # For each table cell
    for item in soup.find_all('div', class_='form-group'):
        if item.input is not None:
            value = item.input.get('value')
            
        elif item.find('option', selected="selected"):
            value = item.find('option', selected="selected").get_text()
            
        elif item.find('textarea'):
            value = item.find('textarea').text
            
        else:
            value = None

        key = item.label.get_text()

        # Append relevant info to dictionary
        if value is not None and value != "":
            content[key] =   value
            content['url'] = url

    return content

In [20]:

def find_links(url, type):
    """
    Items are divided into
    Fasteners (each one is a Fastener)
    Materials (each one is a Material)
    ProcessMultipliers (each one is a Process Multiplier)
    Processes (each one is a Process)
    Tools (each one is a Tool)
    """
    
    # Remove plural from type and create basis URL
    item_type = type[:-2] if type =='Processes' else type[:-1] 
    start = f'https://www.fsaeonline.com/cdsweb/cst/Edit{type}.aspx?CatalogVersion=2132&{item_type}ID='
    end = '&Action=View'

    soup = prepare_soup(url)

    # Get all links
    script = soup.find_all('script', type='text/javascript')[7]
    actions = re.findall("actions: '(.+?)' }", script.string)

    return [f"{start}{action}{end}" for action in actions]

In [15]:
# Does all the processing for each component / process type
def get_data(type):
    url = f"https://www.fsaeonline.com/cst/Browse{type}.aspx"
    
    links = find_links(url, type)
    items = []

    for link in links:    
        item = find_values(link)
        items.append(item)
        sleep(10/1000)
        clear_output(wait=True)
        print(f'Loading {type}... {len(items)}/{len(links)}')
    
    print('End')
    return pd.DataFrame(items)

#### Process information for all types

In [23]:
# Load all info
components = ["Fasteners", "Materials", "ProcessMultipliers", "Processes", "Tools"]
raw_data = {}

for component in components:
    raw_data[component] = get_data(component)

Loading Tools... 10/10
End


#### Process information for specific item

In [ ]:
fasteners_raw = get_data("Fasteners")
fasteners_raw.head()

In [ ]:
materials_raw = get_data('Materials')
materials_raw.head()

In [ ]:
multipliers_raw = get_data('ProcessMultipliers')
multipliers_raw.head()

In [ ]:
processes_raw = get_data('Processes')
processes_raw.head()

In [21]:
tools_raw = get_data('Tools')
tools_raw.head()

Loading Tools... 10/10
End


,Title*,url,Process,Measurement Unit Code,Description,Cost,Internal Notes
0,Brazing Fixture,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Braze,point,Each point is a pickup or support point.,500.00,NaN
1,Die Casting - Die,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Die Casting,die,Per die not die set. Minimum number of dies i...,"10,000.00",NaN
2,Lamination - Flat Panel Tool,https://www.fsaeonline.com/cdsweb/cst/EditTool...,"Lamination, Manual",m^2,Use surface area of tool that is used to form...,"1,500.00",NaN
3,Lamination - Mold Tool,https://www.fsaeonline.com/cdsweb/cst/EditTool...,"Lamination, Manual",m^2,Use surface area of tool that is used to form...,"20,000.00","Replaces 3 tools: Aluminum Tool, Composite To..."
4,Plastic injection molding - Die,https://www.fsaeonline.com/cdsweb/cst/EditTool...,Plastic injection molding,die,Per die not die set.,"10,000.00",NaN


### Saving as csv

In [26]:
# Generate all CSVs
for name, info in raw_data.items():
    info.to_csv(f"{name}_raw.csv", index=False)

In [ ]:
# Generate CSV for specific type
specific_type = fasteners_raw
specific_type.to_csv("Fasteners_raw.csv", index=False)

### Making prettier

In [28]:
# fasteners_raw = pd.read_csv('fasteners_raw.csv')
# materials_raw = pd.read_csv('materials_raw.csv')
# multipliers_raw = pd.read_csv('multipliers_raw.csv')
# processes_raw = pd.read_csv('processes_raw.csv')
# tools_raw = pd.read_csv('tools_raw.csv')

for name, info in raw_data.items():
    raw_data[name] = pd.read_csv(f"{name}_raw.csv")

In [29]:
dfs_raw = raw_data.values()
dfs = []

for df in dfs_raw:
    # Update formulas
    df = df.replace({np.nan: None, "[Not Used]": None})
    dfs.append(df)
    
fasteners, materials, multipliers, processes, tools = dfs

In [30]:
fasteners['Label 1'] = fasteners['Size 1 Label'] + '(' + fasteners['Measurement Unit 1 Code'] + ')'
fasteners['Label 2'] = fasteners['Size 2 Label'] + '(' + fasteners['Measurement Unit 2 Code'] + ')'

map_fasteners = {'Title*' : 'Fastener', 'url' : 'Link', '[C1]' : 'C1', '[C2]': 'C2', ' Cost Formula ': 'Cost Formula'}
fasteners.rename(columns = map_fasteners, inplace = True)
fasteners = fasteners[['Fastener', 'Label 1', 'Label 2', 'Cost Formula', 'C1', 'C2', 'Description', 'Link']]

In [31]:
materials['Label 1'] = materials['Size 1 Label'] + '(' + materials['Measurement Unit 1 Code'] + ')'
materials['Label 2'] = materials['Size 2 Label'] + '(' + materials['Measurement Unit 2 Code'] + ')'

map_materials = {'Title*' : 'Material', 'url' : 'Link', '[C1]' : 'C1', '[C2]': 'C2', ' Cost Formula ': 'Cost Formula'}
materials.rename(columns = map_materials, inplace = True)

materials = materials[['Material', 'Label 1', 'Label 2', 'Cost Formula', 'C1', 'C2', 'Description', 'Supplier', 
                       'Category', 'Link']]

In [32]:
map_multipliers = {'Multiplier Type*' : 'Type', 'url' : 'Link', 'Multiplier Title': 'Multiplier'}
multipliers.rename(columns = map_multipliers, inplace = True)

multipliers = multipliers[['Multiplier', 'Type', 'Multiplier Value', 'Description', 'Link']]

In [33]:
map_processes = {'Title*' : 'Process', 'url' : 'Link', 'Measurement Unit 1 Code' : 'Unit'}
processes.rename(columns = map_processes, inplace = True)

processes = processes[['Process', 'Unit', 'Unit Cost', 'Category', 'Process Multiplier Type','ToolingRequired', 
                       'Near Net Shape', 'Description','Link']]

In [34]:
map_tools = {'Title*' : 'Tool', 'url' : 'Link', 'Measurement Unit Code' : 'Unit'}
tools.rename(columns = map_tools, inplace = True)

tools = tools[['Tool', 'Process', 'Unit', 'Cost', 'Description', 'Link']]

### Final Result

In [35]:
fasteners.head()

,Fastener,Label 1,Label 2,Cost Formula,C1,C2,Description,Link
0,Alcoa Camloc Fastener 1/4 turn,NaN,NaN,4.00,None,None,None,https://www.fsaeonline.com/cdsweb/cst/EditFast...
1,"Bolt, Aluminum",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.0,0.005,Strength 255 Mpa. Special varities included (...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
2,"Bolt, Grade 10.9 (SAE 8)",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.0,0.004,Strength 1030 Mpa. Special varities included ...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
3,"Bolt, Grade 12.9",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",1.25,0.005,Strength 1170 Mpa. Special varities included ...,https://www.fsaeonline.com/cdsweb/cst/EditFast...
4,"Bolt, Grade 6.8 (SAE 3) and All Grades less th...",diameter(mm),length(mm),"[C1]/105154*Math.pow([Size1],2)*[Size2]*Math.s...",0.6,0.0024,Strength 670 Mpa. Special varities included (...,https://www.fsaeonline.com/cdsweb/cst/EditFast...


In [36]:
materials.head()

,Material,Label 1,Label 2,Cost Formula,C1,C2,Description,Supplier,Category,Link
0,"90-Degree Coupler, Tilton 72-560",NaN,NaN,45,None,None,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
1,"90-Degree Coupler, Tilton 72-561",NaN,NaN,47.5,None,None,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
2,"90-Degree Coupler, Tilton 72-562",NaN,NaN,50,None,None,None,Tilton,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
3,"ABS Kit and ESP Module, Bosch, ESP-9",NaN,NaN,500,None,None,ESP-9 module only. Does not include wiring or...,Bosch,Brake System,https://www.fsaeonline.com/cdsweb/cst/EditMate...
4,Accusump,NaN,NaN,([C1]*[Size1]+[C2]),14.3,82.0,"Gage not included, include as separate materi...",Canton Racing Products,Engine,https://www.fsaeonline.com/cdsweb/cst/EditMate...


In [37]:
multipliers.head()

,Multiplier,Type,Multiplier Value,Description,Link
0,Assemble - Length > 0.5m,Assembly,1.25,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
1,Disassemble,Assembly,0.8,To be used when removing parts or fasteners,https://www.fsaeonline.com/cdsweb/cst/EditProc...
2,Fastener Engagement Length > 2D,Fastener Installation,1.25,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
3,Fastener Engagement Length > 4D,Fastener Installation,1.5,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
4,Machine - Hole Length >= 4D,"Drill, Tap",1.5,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...


In [228]:
processes.head()

,Process,Unit,Unit Cost,Category,Process Multiplier Type,ToolingRequired,Near Net Shape,Description,Link
0,Adjustment - Misc.,unit,5.00,Labor,None,No,No,"Chain tension, etc.",https://www.fsaeonline.com/cdsweb/cst/EditProc...
1,Aerosol Apply,m^2,5.25,Labor,None,No,No,"For use with glue, adhesives, paint, etc. Inc...",https://www.fsaeonline.com/cdsweb/cst/EditProc...
2,Annodize,unit,0.00,Labor,None,No,No,It is not necessary to include any cost for a...,https://www.fsaeonline.com/cdsweb/cst/EditProc...
3,Anodizing,cm^2,0.00,None,None,No,None,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...
4,"Assemble, >20 kg, Interference",unit,5.63,Labor,Assembly,No,No,None,https://www.fsaeonline.com/cdsweb/cst/EditProc...


In [229]:
tools.head()

,Tool,Process,Unit,Cost,Description,Link
0,Brazing Fixture,Braze,point,500.00,Each point is a pickup or support point.,https://www.fsaeonline.com/cdsweb/cst/EditTool...
1,Die Casting - Die,Die Casting,die,"10,000.00",Per die not die set. Minimum number of dies i...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
2,Lamination - Flat Panel Tool,"Lamination, Manual",m^2,"1,500.00",Use surface area of tool that is used to form...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
3,Lamination - Mold Tool,"Lamination, Manual",m^2,"20,000.00",Use surface area of tool that is used to form...,https://www.fsaeonline.com/cdsweb/cst/EditTool...
4,Plastic injection molding - Die,Plastic injection molding,die,"10,000.00",Per die not die set.,https://www.fsaeonline.com/cdsweb/cst/EditTool...


### Exporting as Excel Sheet

In [40]:
with pd.ExcelWriter("CostCatalog.xlsx") as writer:  
    fasteners.to_excel(writer, sheet_name='Fasteners', index = False)
    materials.to_excel(writer, sheet_name='Materials', index = False)
    multipliers.to_excel(writer, sheet_name='Multipliers', index = False)
    processes.to_excel(writer, sheet_name='Processes', index = False)
    tools.to_excel(writer, sheet_name='Tools', index = False)